In [175]:
import pandas as pd
import requests
import numpy as np
import time
import requests
import bs4
import json
from pymongo.errors import DuplicateKeyError, CollectionInvalid
from pymongo import MongoClient
#import datetime as dt

In [215]:
client = MongoClient()
# Initiate Database
db = client['nyt_articles']
# Initiate Table
tab = db['table_hilary']
# tab.insert({'hilary': 'cool'})

tab_trump = db['table_trump']
# tab_trump.insert({'trump': 'not cool'})

In [137]:
# def single_query(link, payload):
#     response = requests.get(link, params=payload)
#     if response.status_code != 200:
#          print 'WARNING', response.status_code
#     else:
#          return response.json()

In [179]:
from nytimesarticle import articleAPI
api = articleAPI('74c73309c1052e6aa1785df7cd5cef8c:9:69947183')

In [ ]:
# num_pages = list(np.arange(1, 100, 1))
# print [y for y in num_pages]

In [249]:
for i in range(100):
    articles = api.search(q = 'Donald Trump or Donald J Trump', page = i)
    print i
    for y in range(len(articles['response']['docs'])):
        new_data = articles['response']['docs'][y]
        print articles['response']['docs'][y]['_id']
        new_data['_id'] = articles['response']['docs'][y]['_id']
        try:
            tab_trump.insert(new_data)
        except DuplicateKeyError:
            print 'Peggy doesnt know why its a duplicate.'

0
55fbeebb79881056cf38baa1
Peggy doesnt know why its a duplicate.
55fd8e007988102cb2898e75
Peggy doesnt know why its a duplicate.
55fdd3727988102cb2898eeb
Peggy doesnt know why its a duplicate.
55f19ad07988106f7bdc67a2
Peggy doesnt know why its a duplicate.
55eea3ae7988103b801c2b3e
Peggy doesnt know why its a duplicate.
55ed3aa9798810349b01fa4f
Peggy doesnt know why its a duplicate.
55fb3cc779881056cf38b8c1
Peggy doesnt know why its a duplicate.
55f675297988105f6b72b1dc
Peggy doesnt know why its a duplicate.
55fa36547988102b1902874c
Peggy doesnt know why its a duplicate.
55f92fa67988100a8913c531
Peggy doesnt know why its a duplicate.
1
55facefe7988102b1902894c
Peggy doesnt know why its a duplicate.
55fc0df479881056cf38bb0c
Peggy doesnt know why its a duplicate.
55f832577988106e53a8a052
Peggy doesnt know why its a duplicate.
55fc5e657988100e355b4422
Peggy doesnt know why its a duplicate.
55fa315d7988102b19028747
Peggy doesnt know why its a duplicate.
55fbe49779881056cf38ba86
Peggy doesn

In [235]:
links = db.table_hilary.find({},{'web_url': 1, '_id' : 0})
links_h = []
for i in links:
    links_h.append(str(i['web_url']))
    
with open('url_addresses_hilary','w') as f:
    for item in links_h:
        f.write("%s\n" % item)
    #for i in links_h:
    #f.write(str(i['web_url'] + '\n'))

In [236]:
links = db.tab_trump.find({},{'web_url': 1, '_id' : 0})
links_t = []
for i in links:
    links_t.append(i['web_url'])

with open('url_addresses_trump','w') as f:
    for item in links_h:
        f.write("%s\n" % item)

#### content scraping

In [ ]:
html = requests.get(html2).content
soup = bs4.BeautifulSoup(html, 'html.parser')
body = soup.find_all('p', class_= "story-body-text story-content")
content = '\n'.join([x.get_text() for x in body])
db.articles.insert(content)


In [ ]:
# ADD article content as a field to all entries of comment
db.myCollection.update({}, {$set: {"isOpen": false}}, false, true)

###for getting URLs from MongoDB

In [ ]:
def get_body_text(docs):

    # Grab the url from each document, if it exists, then scrape each url for
    # its body text. If we get any errors along the way, continue on to the
    # next document / url to be scraped.
    result = []
    for d in docs:

        # Make a copy of the doc's dictionary
        doc = d.copy()

        # If there's no url (not sure why this happens sometimes) then ditch it
        if not doc['web_url']:
            continue

        # Scrape the doc's url, return a soup object with the url's text.
        soup = get_soup(doc['web_url'])
        if not soup:
            continue

        # Find all of the paragraphs with the correct class.
        # This class tag is specific to NYT articles.
        body = soup.find_all('p', class_= "story-body-text story-content")
        if not body:
            continue

        # Join the resulting body paragraphs' text (returned in a list).
        doc['body'] = '\n'.join([x.get_text() for x in body])

        print doc['web_url']
        result.append(doc)

    return result